# Do recipes with longer preparation times receive higher ratings?

**Name(s)**: Kareen Farhat and Vani Sahjwani

**Website Link**: https://kareenf04.github.io/seafood_health_analysis/

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import re

import plotly.express as px
pd.options.plotting.backend = 'plotly'

from dsc80_utils import * # Feel free to uncomment and use this.

## Step 1: Introduction

In [2]:
#loading data
recipes_path = Path('data') / 'RAW_recipes.csv'
interactions_path = Path('data') / 'interactions.csv'
recipes = pd.read_csv(recipes_path)
interactions = pd.read_csv(interactions_path)

In [3]:
recipes.head()

,name,id,minutes,contributor_id,...,steps,description,ingredients,n_ingredients
0,1 brownies in the world best ever,333281,40,985201,...,['heat the oven to 350f and arrange the rack i...,"these are the most; chocolatey, moist, rich, d...","['bittersweet chocolate', 'unsalted butter', '...",9
1,1 in canada chocolate chip cookies,453467,45,1848091,...,"['pre-heat oven the 350 degrees f', 'in a mixi...",this is the recipe that we use at my school ca...,"['white sugar', 'brown sugar', 'salt', 'margar...",11
2,412 broccoli casserole,306168,40,50969,...,"['preheat oven to 350 degrees', 'spray a 2 qua...",since there are already 411 recipes for brocco...,"['frozen broccoli cuts', 'cream of chicken sou...",9
3,millionaire pound cake,286009,120,461724,...,"['freheat the oven to 300 degrees', 'grease a ...",why a millionaire pound cake? because it's su...,"['butter', 'sugar', 'eggs', 'all-purpose flour...",7
4,2000 meatloaf,475785,90,2202916,...,"['pan fry bacon , and set aside on a paper tow...","ready, set, cook! special edition contest entr...","['meatloaf mixture', 'unsmoked bacon', 'goat c...",13


In [4]:
interactions.head()

,user_id,recipe_id,date,rating,review
0,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
1,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
2,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."
3,124416,120345,2011-08-06,0,"Just an observation, so I will not rate. I fo..."
4,2000192946,120345,2015-05-10,2,This recipe was OVERLY too sweet. I would sta...


In [5]:
#mergings dfs
merged = recipes.merge(interactions, left_on = 'id', right_on = 'recipe_id', how = 'left')

In [6]:
working = merged.copy()
#nans to 0s
working['rating']=working['rating'].replace(0,np.nan).fillna(np.nan)

#avg rating per receipe
working['rating'] = working.groupby('recipe_id')['rating'].transform('mean')

#list of reviews for each recipe
#working['review'] = working['review'].astype(str)
working = working[['name', 'id', 'minutes', 'tags','nutrition', 'description', \
                   'ingredients','n_ingredients', 'rating']]
working = working.groupby('id').max()
working

,name,minutes,tags,nutrition,description,ingredients,n_ingredients,rating
id,,,,,,,,
275022,impossible macaroni and cheese pie,50,"['60-minutes-or-less', 'time-to-make', 'course...","[386.1, 34.0, 7.0, 24.0, 41.0, 62.0, 8.0]",one of my mom's favorite bisquick recipes. thi...,"['cheddar cheese', 'macaroni', 'milk', 'eggs',...",7,3.0
275024,impossible rhubarb pie,55,"['60-minutes-or-less', 'time-to-make', 'course...","[377.1, 18.0, 208.0, 13.0, 13.0, 30.0, 20.0]",a childhood favorite of mine. my mom loved it ...,"['rhubarb', 'eggs', 'bisquick', 'butter', 'sal...",8,3.0
275026,impossible seafood pie,45,"['60-minutes-or-less', 'time-to-make', 'course...","[326.6, 30.0, 12.0, 27.0, 37.0, 51.0, 5.0]",this is an oldie but a goodie. mom's stand by ...,"['frozen crabmeat', 'sharp cheddar cheese', 'c...",9,3.0
...,...,...,...,...,...,...,...,...
537543,moist gingerbread cake,55,"['60-minutes-or-less', 'time-to-make', 'course...","[1617.0, 104.0, 213.0, 8.0, 40.0, 203.0, 80.0]",a slightly sticky loaf cake flavoured with gin...,"['unsalted butter', 'applesauce', 'egg', 'unsu...",10,NaN
537671,nutcracker peppermint red velvet cake pops,135,"['time-to-make', 'course', 'preparation', 'occ...","[207.9, 12.0, 93.0, 10.0, 6.0, 8.0, 10.0]",rich red velvet cake combines with cool pepper...,"[""devil's food cake mix"", 'eggs', 'buttermilk'...",12,NaN
537716,mini buffalo chicken cheesesteaks,40,"['60-minutes-or-less', 'time-to-make', 'course...","[407.9, 34.0, 21.0, 49.0, 28.0, 64.0, 12.0]",these party-style chicken cheesesteaks are fla...,"['olive oil', 'green bell pepper', 'yellow oni...",13,5.0


In [7]:
#adding calories column
def helper(s):
    return float(s.split(',')[0][1:])
working['calories'] = working['nutrition'].apply(helper)
working

,name,minutes,tags,nutrition,...,ingredients,n_ingredients,rating,calories
id,,,,,,,,,
275022,impossible macaroni and cheese pie,50,"['60-minutes-or-less', 'time-to-make', 'course...","[386.1, 34.0, 7.0, 24.0, 41.0, 62.0, 8.0]",...,"['cheddar cheese', 'macaroni', 'milk', 'eggs',...",7,3.0,386.1
275024,impossible rhubarb pie,55,"['60-minutes-or-less', 'time-to-make', 'course...","[377.1, 18.0, 208.0, 13.0, 13.0, 30.0, 20.0]",...,"['rhubarb', 'eggs', 'bisquick', 'butter', 'sal...",8,3.0,377.1
275026,impossible seafood pie,45,"['60-minutes-or-less', 'time-to-make', 'course...","[326.6, 30.0, 12.0, 27.0, 37.0, 51.0, 5.0]",...,"['frozen crabmeat', 'sharp cheddar cheese', 'c...",9,3.0,326.6
...,...,...,...,...,...,...,...,...,...
537543,moist gingerbread cake,55,"['60-minutes-or-less', 'time-to-make', 'course...","[1617.0, 104.0, 213.0, 8.0, 40.0, 203.0, 80.0]",...,"['unsalted butter', 'applesauce', 'egg', 'unsu...",10,NaN,1617.0
537671,nutcracker peppermint red velvet cake pops,135,"['time-to-make', 'course', 'preparation', 'occ...","[207.9, 12.0, 93.0, 10.0, 6.0, 8.0, 10.0]",...,"[""devil's food cake mix"", 'eggs', 'buttermilk'...",12,NaN,207.9
537716,mini buffalo chicken cheesesteaks,40,"['60-minutes-or-less', 'time-to-make', 'course...","[407.9, 34.0, 21.0, 49.0, 28.0, 64.0, 12.0]",...,"['olive oil', 'green bell pepper', 'yellow oni...",13,5.0,407.9


In [8]:
# ins_list['ingredients'].explode().value_counts().index.map(lambda x : x if len(re.findall(r'peanut',x))>0 else None).dropna()
# #return re.sub(r's\b','',re.sub(r'oes','o',re.sub(r'ies','y',string)))

## Step 2: Data Cleaning and Exploratory Data Analysis

In [9]:
has_allergens =working.copy()
seafood = ['caviar','cod','tuna','bass','salmon','mahi','fish','herring','mackerel',\
           'trout','sardine','shrimp','crab','lobster','calamari','squid','octopus',\
           'mussel','eel','scallop']
has_allergens['has_tag'] = list(map(lambda x: 'seafood' in x or 'crab' in x or 'shellfish'\
                                    in x, working['tags']))
has_allergens['has_ingred'] = list(map(lambda x: sum([ingred in x for ingred in seafood]),\
                                       working['ingredients']))
has_allergens['has_tag'] = has_allergens['has_tag'].astype(int)
#has_allergens['has_ingred'] =has_allergens['has_ingred'].astype(int)
has_allergens['is_sea'] = (has_allergens['has_ingred'] + has_allergens['has_tag']).astype(bool)
has_allergens

,name,minutes,tags,nutrition,...,calories,has_tag,has_ingred,is_sea
id,,,,,,,,,
275022,impossible macaroni and cheese pie,50,"['60-minutes-or-less', 'time-to-make', 'course...","[386.1, 34.0, 7.0, 24.0, 41.0, 62.0, 8.0]",...,386.1,0,0,False
275024,impossible rhubarb pie,55,"['60-minutes-or-less', 'time-to-make', 'course...","[377.1, 18.0, 208.0, 13.0, 13.0, 30.0, 20.0]",...,377.1,0,0,False
275026,impossible seafood pie,45,"['60-minutes-or-less', 'time-to-make', 'course...","[326.6, 30.0, 12.0, 27.0, 37.0, 51.0, 5.0]",...,326.6,1,1,True
...,...,...,...,...,...,...,...,...,...
537543,moist gingerbread cake,55,"['60-minutes-or-less', 'time-to-make', 'course...","[1617.0, 104.0, 213.0, 8.0, 40.0, 203.0, 80.0]",...,1617.0,0,0,False
537671,nutcracker peppermint red velvet cake pops,135,"['time-to-make', 'course', 'preparation', 'occ...","[207.9, 12.0, 93.0, 10.0, 6.0, 8.0, 10.0]",...,207.9,0,0,False
537716,mini buffalo chicken cheesesteaks,40,"['60-minutes-or-less', 'time-to-make', 'course...","[407.9, 34.0, 21.0, 49.0, 28.0, 64.0, 12.0]",...,407.9,0,0,False


In [10]:
heathy = has_allergens.copy()

#drop extra cols
heathy = heathy.drop(columns=['has_ingred', 'has_tag'])

#add sodium in g col
def make_sodium(s):
    return float(s.split(',')[3])*2.3/100 if float(s.split(',')[3])<400 else\
    float(s.split(',')[3])/1000
heathy['sodium'] = heathy['nutrition'].apply(make_sodium)
#heathy

In [11]:
#add protein col in grams
def make_protein(s):
    return float(s.split(',')[4])*50/100 if float(s.split(',')[3])<400 else\
    float(s.split(',')[4])
heathy['protein'] = heathy['nutrition'].apply(make_protein)

#add satfat col
def make_satfat(s):
    return float(s.split(',')[5])*20/100 if float(s.split(',')[3])<400 else \
    float(s.split(',')[5])
heathy['satfat'] = heathy['nutrition'].apply(make_satfat)

#add total fats col
def make_total_fats(s):
    return float(s.split(',')[1])*78/100 if float(s.split(',')[3])<400 else \
    float(s.split(',')[1])
heathy['total_fat'] = heathy['nutrition'].apply(make_total_fats)
#heathy

In [12]:
#standardise columns of nutrition
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
stdised = scaler.fit_transform(heathy[['sodium','protein','satfat','total_fat']])
heathy[['sodium','protein','satfat','total_fat']] = stdised
#heathy

In [13]:
from sklearn.preprocessing import QuantileTransformer, FunctionTransformer

In [14]:
#creating a score for healthy
def custom_transformer(df):
    w, x, y, z = df['sodium'],df['protein'], df['satfat'], df['total_fat']
    return 1- (w*y)/(x*z)
      
transformer = FunctionTransformer(custom_transformer)
heathy['health_score'] = transformer.transform(heathy[['sodium','protein','satfat',\
                                                       'total_fat']])
#heathy

In [15]:
# calculating quantiles of scores so that we know what is relatively healthy and what isn't
tiles = QuantileTransformer(n_quantiles = 100).fit(heathy[['health_score']])
heathy['score_quantile'] = tiles.transform(heathy[['health_score']])*100

#discretising quantiles
from sklearn.preprocessing import KBinsDiscretizer
kbins = KBinsDiscretizer(n_bins=100, encode='ordinal', strategy='uniform') # other strategies are 'quantile' and 'kmeans'
heathy['binned_quantiles'] = kbins.fit_transform(heathy[['score_quantile']])
heathy.columns

Index(['name', 'minutes', 'tags', 'nutrition', 'description', 'ingredients',
       'n_ingredients', 'rating', 'calories', 'is_sea', 'sodium', 'protein',
       'satfat', 'total_fat', 'health_score', 'score_quantile',
       'binned_quantiles'],
      dtype='object')

In [16]:
heathy['health_score'].head()

id
275022    2.08
275024    0.69
275026    2.38
275030    0.77
275032    0.35
Name: health_score, dtype: float64

In [17]:
#frequency of scores
px.histogram(heathy,x='score_quantile',nbins = 25,title='Count of Scores'\
            ).update_layout(xaxis_title="Score", yaxis_title="Count")

In [18]:
#avg rating per number of ingredients
heathy['avg_rating_nins'] = heathy.groupby('n_ingredients')['rating'].transform('mean')
px.scatter(heathy, x='n_ingredients',y='avg_rating_nins', \
           title='Avg Rating per Number of Ingredients').update_layout(\
    xaxis_title="Number of ingredients", yaxis_title="Average rating")

In [19]:
#avg health score per number of ingredients
heathy['avg_score_nins'] = heathy.groupby('n_ingredients')['score_quantile'].transform('mean')
fig = px.scatter(heathy,x='n_ingredients',y='avg_score_nins',\
                 title='Average Health Score per Number of Ingredients').update_layout(\
    xaxis_title="Number of ingredients", yaxis_title="Average health score")
fig.update_layout(height=400)
# fig.write_html('seafood_health_analysis/assets/univariate.html', include_plotlyjs='cdn')
fig.show()

In [20]:
#avg number of ingredients per health score
heathy['avg_n_ins'] = heathy.groupby('binned_quantiles')['n_ingredients'].transform('mean')
heathy.plot(kind = 'scatter',y='avg_n_ins',x='binned_quantiles',\
            title='Average Number of Ingredients per Health Score').update_layout(\
    xaxis_title="Health score", yaxis_title="Average number of ingredients")

In [21]:
#discretising ratings
heathy = heathy[heathy['rating'].notna()].drop(columns =\
                                               ['avg_rating_nins','avg_score_nins','avg_n_ins'])
kbins = KBinsDiscretizer(n_bins=6, encode='ordinal', strategy='uniform')
heathy['binned_ratings'] = kbins.fit_transform(heathy[['rating']])
heathy

,name,minutes,tags,nutrition,...,health_score,score_quantile,binned_quantiles,binned_ratings
id,,,,,,,,,
275022,impossible macaroni and cheese pie,50,"['60-minutes-or-less', 'time-to-make', 'course...","[386.1, 34.0, 7.0, 24.0, 41.0, 62.0, 8.0]",...,2.08,89.45,89.0,3.0
275024,impossible rhubarb pie,55,"['60-minutes-or-less', 'time-to-make', 'course...","[377.1, 18.0, 208.0, 13.0, 13.0, 30.0, 20.0]",...,0.69,57.51,57.0,3.0
275026,impossible seafood pie,45,"['60-minutes-or-less', 'time-to-make', 'course...","[326.6, 30.0, 12.0, 27.0, 37.0, 51.0, 5.0]",...,2.38,90.92,90.0,3.0
...,...,...,...,...,...,...,...,...,...
537459,bailey s chocotini,10,"['15-minutes-or-less', 'time-to-make', 'course...","[220.7, 15.0, 49.0, 2.0, 3.0, 30.0, 4.0]",...,0.66,56.36,56.0,5.0
537485,5 ingredient salted caramel crumble bars,45,"['60-minutes-or-less', 'time-to-make', 'course...","[52.8, 3.0, 0.0, 4.0, 1.0, 1.0, 2.0]",...,0.42,41.44,41.0,5.0
537716,mini buffalo chicken cheesesteaks,40,"['60-minutes-or-less', 'time-to-make', 'course...","[407.9, 34.0, 21.0, 49.0, 28.0, 64.0, 12.0]",...,49.37,99.05,99.0,5.0


In [22]:
#avg score per rating
grouped_data = heathy.groupby(['binned_ratings', 'is_sea'])[['score_quantile']].mean().reset_index()
grouped_data

import plotly.graph_objects as go
heathy_true = grouped_data[grouped_data['is_sea']]
heathy_false = grouped_data[grouped_data['is_sea']==False]

fig = go.Figure(data=[
    go.Bar(name='True', x=heathy_true['binned_ratings'], y=heathy_true['score_quantile']),
    go.Bar(name='False', x=heathy_false['binned_ratings'], y=heathy_false['score_quantile'])
])
# Change the bar mode
fig.update_layout(barmode='group').update_layout(\
    xaxis_title="Rating", yaxis_title="Average health score", \
    legend_title_text="Has seafood", \
    title="Average Health Score per Rating of Recipes<br>With and Without Seafood")
fig.update_layout(height=400)
fig.show()

In [23]:
#avg rating per score for seafood and not seafood
grouped = heathy.groupby(['is_sea','binned_quantiles'])['rating'].mean().reset_index()
issea = grouped[grouped['is_sea']]
notsea = grouped[grouped['is_sea']==False]

fig = go.Figure(data=[
    go.Scatter(name='True', x=issea['binned_quantiles'], y=issea['rating']),
    go.Scatter(name='False', x=notsea['binned_quantiles'], y=notsea['rating'])
])
# Change the bar mode
fig.update_layout(scattermode='group').update_layout(\
    xaxis_title="Health score", \
    yaxis_title="Average rating", legend_title_text="Has seafood", \
    title="Average Rating per Health Score of Recipes<br>With and Without Seafood")
fig.show()

In [24]:
#avg time to prep VS rating
grouped_data = heathy.groupby(['binned_ratings', 'is_sea'])\
[['minutes']].mean().reset_index()#.set_index('binned_ratings')#.unstack()
grouped_data

import plotly.graph_objects as go
heathy_true = grouped_data[grouped_data['is_sea']]
heathy_false = grouped_data[grouped_data['is_sea']==False]

fig = go.Figure(data=[
    go.Bar(name='True', x=heathy_true['binned_ratings'], y=heathy_true['minutes']),
    go.Bar(name='False', x=heathy_false['binned_ratings'], y=heathy_false['minutes'])
])
# Change the bar mode
fig.update_layout(barmode='group').update_layout(\
    xaxis_title="Rating", yaxis_title="Average prep time (minutes)",\
    legend_title_text="Has seafood", \
    title="Average Prep Time per Rating of Recipes<br>With and Without Seafood")
fig.show()

In [25]:
#minutes less than 180
less_time = heathy[heathy['minutes']<=180]
less_time['mean'] = less_time['score_quantile']
less_time['median'] = less_time['score_quantile']
less_time['max'] = less_time['score_quantile']
less_time['min'] = less_time['score_quantile']

#avg time to prep VS score pivot table
agged = less_time.groupby(['is_sea','minutes']).agg(\
    {'mean':'mean','median':'median','max':'max','min':'min'}).reset_index()

#sep sea and not
agg_sea = agged[agged['is_sea']]
agg_not = agged[agged['is_sea']==False]

agged.head()

,is_sea,minutes,mean,median,max,min
0,False,0,77.66,77.66,77.66,77.66
1,False,1,40.96,36.36,99.00,1.00
2,False,2,41.08,36.36,99.19,1.00
3,False,3,40.30,36.36,99.78,1.00
4,False,4,40.59,36.36,98.27,1.00


In [26]:
#plotting aggs like mean, median, max, min VS score for seafood
fig = go.Figure(data=[
    go.Scatter(name='Mean', x=agg_sea['minutes'], y=agg_sea['mean']),
    go.Scatter(name='Median', x=agg_sea['minutes'], y=agg_sea['median']),
    go.Scatter(name='Max', x=agg_sea['minutes'], y=agg_sea['max']),
    go.Scatter(name='Min', x=agg_sea['minutes'], y=agg_sea['min'])
])
# Change the bar mode
fig.update_layout(scattermode='group').update_layout(\
    xaxis_title="Prep time (minutes)", yaxis_title="Health score",\
    legend_title_text="Aggregate function", \
    title={'text':"Aggregate Functions vs Prep Time<br>of Recipes With Seafood", 'x': 0.425})
fig.update_layout(height=400, width=700)
fig.show()
# fig.write_html('seafood_health_analysis/assets/agg_sea.html', include_plotlyjs='cdn')

In [27]:
#plotting aggs like mean, median, max, min VS score for non seafood
fig = go.Figure(data=[
    go.Scatter(name='Mean', x=agg_not['minutes'], y=agg_not['mean']),
    go.Scatter(name='Median', x=agg_not['minutes'], y=agg_not['median']),
    go.Scatter(name='Max', x=agg_not['minutes'], y=agg_not['max']),
    go.Scatter(name='Min', x=agg_not['minutes'], y=agg_not['min'])
])
# Change the bar mode
fig.update_layout(scattermode='group').update_layout(\
    xaxis_title="Prep time (minutes)", yaxis_title="Health score", \
    legend_title_text="Aggregate function",\
    title={'text':"Aggregate Functions vs Prep Time<br>of Recipes Without Seafood", 'x': 0.425})
fig.update_layout(height=400, width=700)
fig.show()
# fig.write_html('seafood_health_analysis/assets/agg_notsea.html', include_plotlyjs='cdn')

## Step 3: Assessment of Missingness

In [28]:
df = heathy.copy()

In [29]:
df['description'].isna().sum()

np.int64(66)

In [30]:
np.random.seed(8346)
df['missing_description'] = df['description'].isna().astype(int)

# Calculate observed difference in missingness between seafood and non-seafood recipes
observed_difference = abs(df[df['is_sea'] == 1]['missing_description'].mean() - \
                      df[df['is_sea'] == 0]['missing_description'].mean())

# Permutation test
num_permutations = 5000
differences = []

for _ in range(num_permutations):
    shuffled_df = df.assign(shuffled_missing_description=np.random.permutation(df['missing_description']))
    # shuffled_missing = df['missing_description'].sample(frac=1, replace=False).reset_index(drop=True)
    # shuffled_df = df.copy()
    # shuffled_df['shuffled_missing_description'] = shuffled_missing

    # Calculate difference in means
    diff = abs(shuffled_df[shuffled_df['is_sea'] == 1]['shuffled_missing_description'].mean() - \
           shuffled_df[shuffled_df['is_sea'] == 0]['shuffled_missing_description'].mean())
    
    differences.append(diff)

# Compute p-value
p_value = np.mean(np.array(differences) >= observed_difference)

print(f"Observed Difference: {observed_difference}")
print(f"P-value: {p_value}")

Observed Difference: 0.00022015601476858793
P-value: 0.6612


In [31]:
fig = px.histogram(
    x=differences, 
    nbins=24, 
    title="Empirical Distribution",
    labels={'x': "Difference in proportions", 'y': "Frequency"}
)

# Add observed statistic as a vertical line
fig.add_vline(
    x=observed_difference,
    line_width=2,
    line_dash="solid", 
    line_color="red",
    opacity = 1,
    annotation_text=f"observed statictic<br>= {observed_difference:.4f}",
    annotation_font=dict(size=12, color="red", family="Arial"),
    annotation_position="top right"
)
fig.update_traces(marker=dict(line=dict(width=0)))
fig.update_layout(height=450)

# Show the figure
fig.show()
# fig.write_html('seafood_health_analysis/assets/missing2.html', include_plotlyjs='cdn')

In [32]:
observed_difference_2 = abs(df[df['missing_description'] == 1]['health_score'].mean() - \
                      df[df['missing_description'] == 0]['health_score'].mean())

# Permutation test
num_permutations = 5000
differences_2 = []

for _ in range(num_permutations):
    shuffled_df = df.assign(shuffled_missing_description=np.random.permutation(df['missing_description']))

    diff = abs(shuffled_df[shuffled_df['shuffled_missing_description'] == 1]['health_score'].mean() - \
           shuffled_df[shuffled_df['shuffled_missing_description'] == 0]['health_score'].mean())
    
    differences_2.append(diff)

# Compute p-value
p_value_2 = np.mean(np.array(differences_2) >= observed_difference_2)

print(f"Observed Difference: {observed_difference_2}")
print(f"P-value: {p_value_2}")

Observed Difference: 0.6964693878363533
P-value: 0.5168


In [33]:
fig = px.histogram(
    x=differences_2, 
    nbins=100, 
    title="Empirical Distribution",
    labels={'x': "Difference in means", 'y': "Frequency"}
)

# Add observed statistic as a vertical line
fig.add_vline(
    x=observed_difference,
    line_width=2,
    line_dash="solid", 
    line_color="red",
    opacity = 1,
    annotation_text=f"observed statictic<br>= {observed_difference_2:.4f}",
    annotation_font=dict(size=12, color="red", family="Arial"),
    annotation_position="top right"
)
fig.update_traces(marker=dict(line=dict(width=0)))
fig.update_layout(height=450)

# Show the figure
fig.show()
# fig.write_html('seafood_health_analysis/assets/missing1.html', include_plotlyjs='cdn')

## Step 4: Hypothesis Testing

In [34]:
obs_score = heathy[heathy['is_sea']]['score_quantile'].mean() - heathy[heathy['is_sea']==False]['score_quantile'].mean()
obs_rate = issea['rating'].mean() - notsea['rating'].mean()

In [35]:
#perm test for score
N = 1000
stats = []
for i in range(N):
    with_shuffled = heathy.copy()
    with_shuffled['is_sea'] = np.random.permutation(with_shuffled['is_sea'])
    stats.append(with_shuffled[with_shuffled['is_sea']]['score_quantile'].mean() - \
                 with_shuffled[with_shuffled['is_sea']==False]['score_quantile'].mean())
p_val = (np.array(stats)>=obs_score).mean()
p_val


np.float64(0.0)

In [36]:
fig = px.histogram(pd.DataFrame(stats), x=0, nbins=20, histnorm='probability', 
                   title='Empirical Distribution of the Difference<br>in Average Health Scores').update_layout(
                    xaxis_title="Difference in average health score")
fig.add_vline(x=obs_score,
            line_width=3,
            line_dash="solid", 
            line_color="red",
            opacity = 1,
            annotation_text=f"observed statistic<br>= {obs_score:.4f}",
            annotation_font=dict(size=12, color="red", family="Arial"),
            annotation_position="top right")
fig.update_layout(height=400)
fig.show()
# fig.write_html('seafood_health_analysis/assets/hyp1.html', include_plotlyjs='cdn')

In [37]:
#perm test for score
N2 = 1000
stats2 = []
no_nans = heathy[heathy['rating'].notna()]
for i in range(N2):
    with_shuffled2 = heathy.copy()
    with_shuffled2['is_sea'] = np.random.permutation(with_shuffled2['is_sea'])
    stats2.append(with_shuffled2[with_shuffled2['is_sea']]['rating'].mean() - \
                  with_shuffled2[with_shuffled2['is_sea']==False]['rating'].mean())
p_val2 = (np.array(stats2)>=obs_rate).mean()
p_val2

np.float64(0.041)

In [38]:
fig2 = px.histogram(pd.DataFrame(stats2), nbins=20, histnorm='probability', 
                   title='Empirical Distribution of the Difference<br>in Average Ratings').update_layout(\
    xaxis_title="Difference in average rates")
fig2.add_vline(x=obs_rate,
              line_width=3,
            line_dash="solid", 
            line_color="red",
            opacity = 1,
            annotation_text=f"observed statistic<br>= {obs_rate:.4f}",
            annotation_font=dict(size=12, color="red", family="Arial"),
            annotation_position="top right")
fig2.update_layout(height=400, showlegend=False, title=dict(
        # text="Raised Title Example", 
        y=0.967,
        x=0.5,
        xanchor="center",
        yanchor="top"
    ))
fig2.show()
#fig.write_html('seafood_health_analysis/assets/hyp2.html', include_plotlyjs='cdn')

## Step 5: Framing a Prediction Problem

In [39]:
# predicting is_sea based on other columns
heathy[['name', 'minutes', 'tags', 'nutrition', 'description', 'ingredients',
       'n_ingredients', 'rating', 'calories', 'is_sea', 'sodium', 'protein', 'score_quantile']]

,name,minutes,tags,nutrition,...,is_sea,sodium,protein,score_quantile
id,,,,,,,,,
275022,impossible macaroni and cheese pie,50,"['60-minutes-or-less', 'time-to-make', 'course...","[386.1, 34.0, 7.0, 24.0, 41.0, 62.0, 8.0]",...,False,-0.02,0.11,89.45
275024,impossible rhubarb pie,55,"['60-minutes-or-less', 'time-to-make', 'course...","[377.1, 18.0, 208.0, 13.0, 13.0, 30.0, 20.0]",...,False,-0.34,-0.31,57.51
275026,impossible seafood pie,45,"['60-minutes-or-less', 'time-to-make', 'course...","[326.6, 30.0, 12.0, 27.0, 37.0, 51.0, 5.0]",...,True,0.06,0.05,90.92
...,...,...,...,...,...,...,...,...,...
537459,bailey s chocotini,10,"['15-minutes-or-less', 'time-to-make', 'course...","[220.7, 15.0, 49.0, 2.0, 3.0, 30.0, 4.0]",...,False,-0.66,-0.46,56.36
537485,5 ingredient salted caramel crumble bars,45,"['60-minutes-or-less', 'time-to-make', 'course...","[52.8, 3.0, 0.0, 4.0, 1.0, 1.0, 2.0]",...,False,-0.60,-0.49,41.44
537716,mini buffalo chicken cheesesteaks,40,"['60-minutes-or-less', 'time-to-make', 'course...","[407.9, 34.0, 21.0, 49.0, 28.0, 64.0, 12.0]",...,False,0.70,-0.09,99.05


## Step 6: Baseline Model

In [40]:
#splitting data into training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(\
    heathy[['minutes','rating','score_quantile','calories','n_ingredients',\
            'protein','sodium','satfat','total_fat','tags']],heathy['is_sea'], random_state=100)

In [41]:
#relevant imports
from sklearn.metrics import root_mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [42]:
from sklearn.tree import DecisionTreeClassifier

In [43]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import Binarizer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

#pipeline to standardise minutes and create a decision tree with only criterion specified
preproc = make_column_transformer((StandardScaler(), ['minutes']),remainder='passthrough',)
pl = make_pipeline(preproc, DecisionTreeClassifier(max_depth=None, criterion='entropy'))
pl.fit(X_train.drop(columns = ['tags','satfat','protein','total_fat','sodium',\
                               'n_ingredients','rating']), y_train)

/Users/vani/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/compose/_column_transformer.py:1623: FutureWarning:


The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).




Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['minutes'])])),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(criterion='entropy'))])

In [44]:
#measuring performance of this pipeline
train_score = (pl.score(X_train.drop(columns = ['tags','satfat','protein','total_fat',\
                                                'sodium','n_ingredients','rating']), y_train))
test_score = (pl.score(X_test.drop(columns = ['tags','satfat','protein','total_fat',\
                                              'sodium','n_ingredients','rating']), y_test))

train_score, test_score

(0.9999342958984214, 0.8591701980881049)

## Step 7: Final Model

In [49]:
from sklearn.tree import plot_tree

In [50]:
#testing different max depths for the baseline to correct the overfitting
train_score = []
test_score = []

for m in list(list(range(1,30,1))+[None]):
    dt1 = DecisionTreeClassifier(max_depth=m, criterion='entropy')
    dt1.fit(X_train.drop(columns = ['tags','satfat','protein','total_fat',\
                                    'sodium','n_ingredients','rating']), y_train)
    train_score.append(dt1.score(X_train.drop(columns = ['tags','satfat',\
                                                         'protein','total_fat','sodium',\
                                                         'n_ingredients','rating']), y_train))
    test_score.append(dt1.score(X_test.drop(columns = ['tags','satfat','protein',\
                                                       'total_fat','sodium','n_ingredients',\
                                                       'rating']), y_test))

In [51]:
#df of results of errors of varying max depths calculated by 1-score
fig_df = pd.DataFrame()
fig_df['train_score'] = 1-np.array(train_score)
fig_df['test_score']= 1-np.array(test_score)
fig_df['index'] = list(list(range(1,30,1))+['None'])
fig_df = fig_df.set_index('index')
fig_df

,train_score,test_score
index,,
1,8.08e-02,0.08
2,8.08e-02,0.08
3,8.08e-02,0.08
...,...,...
28,2.30e-02,0.13
29,2.02e-02,0.13
None,6.57e-05,0.14


In [52]:
#visualising results to guage a range for max depth
fig = px.line(fig_df)
fig.update_layout(showlegend=True, xaxis_title='Polynomial Degree', yaxis_title='score')

In [53]:
#features of baseline
dt1.feature_names_in_

array(['minutes', 'score_quantile', 'calories'], dtype=object)

In [54]:
#importances of features of baseline
dt1.feature_importances_

array([0.13, 0.44, 0.42])

In [55]:
#measuring performance of this pipeline
train_score = (dt1.score(X_train.drop(columns = ['tags','satfat','protein','total_fat',\
                                                'sodium','n_ingredients','rating']), y_train))
test_score = (dt1.score(X_test.drop(columns = ['tags','satfat','protein','total_fat',\
                                              'sodium','n_ingredients','rating']), y_test))

train_score, test_score

(0.9999342958984214, 0.8606977431753228)

In [52]:
from sklearn.model_selection import GridSearchCV

In [53]:
#hyperparameters of decision tree
hyperparameters = {
    'max_depth': [3, 4,5, 6,7,8,20, None], 
    'min_samples_split': [2, 10, 50, 100, 500, 1000, 2000],
    'criterion': ['gini', 'entropy']
}

In [54]:
#using gridsearch
searcher = GridSearchCV(DecisionTreeClassifier(), hyperparameters, cv=5)

In [55]:
%%time
searcher.fit(X_train.drop(columns = 'tags'), y_train)

CPU times: user 1min 9s, sys: 746 ms, total: 1min 10s
Wall time: 1min 10s


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 4, 5, 6, 7, 8, 20, None],
                         'min_samples_split': [2, 10, 50, 100, 500, 1000,
                                               2000]})

In [56]:
#grid search results
searcher.best_params_

{'criterion': 'gini', 'max_depth': 6, 'min_samples_split': 500}

In [57]:
#new train score
searcher.score(X_train.drop(columns = 'tags'), y_train)

0.9212043561819346

In [58]:
#new test score
searcher.score(X_test.drop(columns = 'tags'), y_test)

0.9207154824085937

In [59]:
#importance of features in final model
dts = DecisionTreeClassifier(**searcher.best_params_)
dts.fit(X_train.drop(columns = 'tags'), y_train)
dts.feature_importances_

array([0.19, 0.  , 0.02, 0.06, 0.05, 0.42, 0.01, 0.24, 0.01])

In [60]:
dts.feature_names_in_

array(['minutes', 'rating', 'score_quantile', 'calories', 'n_ingredients',
       'protein', 'sodium', 'satfat', 'total_fat'], dtype=object)

## Step 8: Fairness Analysis

In [61]:
#binarizing minutes based on threshold of 70
#70 due to observation from agg graphs
new_heathy = heathy.copy()
binar = Binarizer(threshold=70)
new_heathy['binar_minutes'] = binar.transform(heathy[['minutes']])

/Users/vani/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning:

X has feature names, but Binarizer was fitted without feature names



In [62]:
#score of model on data where minutes is more than 70
more = searcher.score(new_heathy[new_heathy['binar_minutes']==1][['minutes',\
                                                                  'rating','score_quantile',\
                                                                  'calories','n_ingredients','protein',\
                                                         'sodium','satfat',\
                                                                  'total_fat']],new_heathy[new_heathy[\
                      'binar_minutes']==1]['is_sea'])
more

0.9538053533982978

In [63]:
#score of model on data where minutes is less than or equal to 70
less = searcher.score(new_heathy[new_heathy['binar_minutes']==0][[\
    'minutes',\
    'rating','score_quantile','calories','n_ingredients','protein',\
    'sodium','satfat','total_fat']],new_heathy[new_heathy['binar_minutes']==0]['is_sea'])
less

0.9129142997890978

In [64]:
#observed test stat = diff between scores
observed = more-less
observed

0.04089105360919998

In [65]:
#running the permutation test
#null hypothesis is that minutes shouldnt affect the accuracy of the model
#alternate: minutes do affect the performance 
N3 = 1000
stats3 = []
for i in range(N3):
    with_shuffled3 = new_heathy.copy()
    with_shuffled3['binar_minutes'] = np.random.permutation(with_shuffled3['binar_minutes'])
    more = searcher.score(with_shuffled3[with_shuffled3['binar_minutes']==1][[\
        'minutes','rating','score_quantile','calories','n_ingredients','protein',\
                                                         'sodium','satfat',\
        'total_fat']],with_shuffled3[with_shuffled3['binar_minutes']==1]['is_sea'])
    less = searcher.score(with_shuffled3[with_shuffled3['binar_minutes']==0][[\
        'minutes','rating','score_quantile','calories','n_ingredients','protein',\
                                                         'sodium',\
        'satfat','total_fat']],with_shuffled3[with_shuffled3['binar_minutes']==0]['is_sea'])
    stats3.append(more - less)
p_val3 = (np.array(stats3)>=observed).mean()
p_val3

np.float64(0.0)

In [68]:
#visualization of p-val < 0.05 therefore reject null
fig3 = px.histogram(pd.DataFrame(stats3), nbins=20, histnorm='probability', 
                    title='Empirical Distribution of the Difference in Health Scores of<br>Recipes with Minutes More Than VS Less Than 70')
fig3.add_vline(x=observed,
            line_width=3,
            line_dash="solid", 
            line_color="red",
            opacity = 1,
            annotation_text=f"observed statistic<br>= {observed:.4f}",
            annotation_font=dict(size=12, color="red", family="Arial"),
            annotation_position="top right")
fig3.update_layout(height=400)
fig3.update_layout(showlegend=False, xaxis_title="Difference in health scores")
fig3.show()
#fig3.write_html('seafood_health_analysis/assets/fair.html', include_plotlyjs='cdn')